# Data Collector and Databse Update

## Library Imports

In [1]:
!pip install --upgrade wbdata

!pip install git+https://github.com/OliverSherouse/wbdata.git

# Install necessary packages
!pip install geopandas
!pip install shapely fiona pyproj
!pip install wbdata pandas_datareader
!pip install pycountry
!pip install geopandas pandas shapely
!pip install qrcode

# Import necessary libraries
import qrcode
from PIL import Image
import io   

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.patches as patches
import random

import pandas_datareader.data as web
import pandas as pd
import geopandas as gpd
import datetime
import pycountry
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
import numpy as np
import statsmodels.api as sm
import os

import datetime
import time

import wbdata
import datetime
import pandas as pd
import time
import requests
from http.client import RemoteDisconnected

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/OliverSherouse/wbdata.git to /tmp/pip-req-build-qxa3we8x
  Running command git clone --filter=blob:none --quiet https://github.com/OliverSherouse/wbdata.git /tmp/pip-req-build-qxa3we8x
  Resolved https://github.com/OliverSherouse/wbdata.git to commit c61d147d62f431b55a896238919cb31fc3766ddd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeabl

Key '4905138695389287408' not in persistent cache.
Key '76379287156053244' not in persistent cache.
Key '5516572655353123816' not in persistent cache.
Key '3678725171189301664' not in persistent cache.
Key '1696867647050300001' not in persistent cache.
Key '7792491775534859921' not in persistent cache.
Key '8002777476547068859' not in persistent cache.
Key '2149127761755539084' not in persistent cache.
Key '1132780383477308786' not in persistent cache.
Key '-1095406477998457869' not in persistent cache.
Key '4689979080496617843' not in persistent cache.
Key '-3586372396234812743' not in persistent cache.
Key '4448609770562540984' not in persistent cache.
Key '-1898861983193954595' not in persistent cache.
Key '4524039279937806392' not in persistent cache.
Key '-4526456258047053706' not in persistent cache.
Key '2303626556744168841' not in persistent cache.
Key '-5673491648173419944' not in persistent cache.
Key '-5348299940131295949' not in persistent cache.
Key '6776093665984234928' n

## Functions

In [2]:
####################################
### Fetcher
####################################

def fetch_data(indicators, countries):
    data = {}
    for indicator_code, indicator_name in indicators.items():
        try:
            print(f"🔎 Fetching: {indicator_name} ({indicator_code})")
            df = wbdata.get_dataframe({indicator_code: indicator_name}, country=countries)
            if df is not None and not df.empty:
                if df.index.name == 'date':
                    df.index = pd.to_datetime(df.index)
                data[indicator_code] = df
                print(f"✅ Success: {indicator_name}")
            else:
                data[indicator_code] = None
                print(f"⚠️ No data for {indicator_name}")
        except Exception as e:
            data[indicator_code] = None
            print(f"❌ Error fetching {indicator_name} ({indicator_code}): {e}")
        time.sleep(1)
    return data


###############################
# merger
################################

def merge_indicator_dict(indicator_dict):
    """
    Merge a dictionary of DataFrames (key=indicator_code, value=DataFrame)
    into one DataFrame by joining on index (usually country, date).
    """
    dfs = [df for df in indicator_dict.values() if df is not None and not df.empty]

    if not dfs:
        return pd.DataFrame()  # return empty if no valid dataframes

    # Merge all dataframes stepwise on their index (usually MultiIndex country, date)
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = merged_df.join(df, how='outer', lsuffix='_left', rsuffix='_right')

    # Reset index to get 'country' and 'date' as columns (if MultiIndex)
    merged_df = merged_df.reset_index()

    return merged_df


## Dictionaries

In [3]:
#######################
# Areas of Interest
#################################33

# Define the countries of interest
ecowas_countries = [
    "BEN", "BFA", "CPV", "CIV", "GMB", "GHA", "GIN", "GNB", 
    "LBR", "MLI", "NER", "NGA", "SEN", "SLE", "TGO"
]




# North Africa
African_countries_NA = [
    "DZA",  # Algeria
    "EGY",  # Egypt
    "LBY",  # Libya
    "MAR",  # Morocco
    "SDN",  # Sudan
    "TUN"   # Tunisia
]

# West Africa (ECOWAS + Mauritania)
African_countries_WA = [
    "BEN", "BFA", "CPV", "CIV", "GMB", "GHA", "GIN", "GNB", 
    "LBR", "MLI", "MRT", "NER", "NGA", "SEN", "SLE", "TGO"
]

# Central Africa
African_countries_CA = [
    "AGO",  # Angola
    "CMR",  # Cameroon
    "CAF",  # Central African Republic
    "TCD",  # Chad
    "COG",  # Republic of Congo
    "COD",  # Democratic Republic of the Congo
    "GNQ",  # Equatorial Guinea
    "GAB",  # Gabon
    "STP"   # São Tomé and Príncipe
]

# East Africa
African_countries_EA = [
    "BDI",  # Burundi
    "COM",  # Comoros
    "DJI",  # Djibouti
    "ERI",  # Eritrea
    "ETH",  # Ethiopia
    "KEN",  # Kenya
    "MDG",  # Madagascar
    "MWI",  # Malawi
    "RWA",  # Rwanda
    "SOM",  # Somalia
    "SSD",  # South Sudan
    "TZA",  # Tanzania
    "UGA"   # Uganda
]

# Southern Africa
African_countries_SA = [
    "BWA",  # Botswana
    "LSO",  # Lesotho
    "MOZ",  # Mozambique
    "NAM",  # Namibia
    "ZAF",  # South Africa
    "ZMB",  # Zambia
    "ZWE",  # Zimbabwe
    "SZL",  # Eswatini (Swaziland)
    "MUS",  # Mauritius
    "SYC"   # Seychelles
]

African_country_groups = {
    "NA": African_countries_NA,
    "WA": African_countries_WA,
    "CA": African_countries_CA,
    "EA": African_countries_EA,
    "SA": African_countries_SA
}




world = ["WLD"]


#####################################################################################################

###############################################
# Indicators
################################################
economic_indicators = {
    "NY.GDP.MKTP.CD": "GDP (current US$)",
    "NY.GDP.PCAP.CD": "GDP per capita (current US$)",
    "NY.GDP.MKTP.KD.ZG": "GDP growth (annual %)",
    "FP.CPI.TOTL": "Inflation, consumer prices (annual %)",
    "NE.EXP.GNFS.ZS": "Exports of goods and services (% of GDP)",
    "NE.IMP.GNFS.ZS": "Imports of goods and services (% of GDP)",
    "BN.CAB.XOKA.GD.ZS": "Current account balance (% of GDP)",
    "BX.KLT.DINV.WD.GD.ZS": "Foreign direct investment, net inflows (% of GDP)",
    "NY.GNS.ICTR.ZS": "Gross savings (% of GDP)",
    "SL.TLF.CACT.ZS": "Labor force participation rate, total (% of total population ages 15+)",
    "NV.AGR.TOTL.ZS": "Agriculture, forestry, and fishing, value added (% of GDP)",
    "NV.IND.TOTL.ZS": "Industry (including construction), value added (% of GDP)",
    "NV.SRV.TOTL.ZS": "Services, value added (% of GDP)",
    "GC.REV.XGRT.GD.ZS": "Revenue, excluding grants (% of GDP)",
    "GC.XPN.TOTL.GD.ZS": "Expense (% of GDP)",
    "GC.NLD.TOTL.GD.ZS": "Net lending (+) / net borrowing (-) (% of GDP)",
    "BX.GSR.MRCH.CD": "Merchandise exports (current US$)",
    "BM.GSR.MRCH.CD": "Merchandise imports (current US$)",
    "NE.CON.PRVT.ZS": "Household final consumption expenditure (% of GDP)",
    "NE.CON.GOVT.ZS": "Government final consumption expenditure (% of GDP)",
    "NY.GDP.DEFL.KD.ZG": "GDP deflator (annual %)",
    "NE.GDI.TOTL.ZS": "Gross fixed capital formation (% of GDP)",
    "NY.GDP.PCAP.PP.CD": "GDP per capita, PPP (current international $)",
    "NY.GDP.MKTP.PP.CD": "GDP, PPP (current international $)",
    "FI.RES.TOTL.CD": "Total reserves (includes gold, current US$)",
    "FI.RES.XGLD.CD": "Total reserves minus gold (current US$)",
    "FI.RES.TOTL.MO": "Total reserves in months of imports",
    "NY.GDP.FCST.PC.GD.ZS": "Forecasted GDP growth rate",
    "NE.CON.TOTL.KD.ZG": "Final consumption expenditure (annual % growth)",
    "NE.CON.PRVT.KD.ZG": "Household consumption (annual % growth)",
    "GC.TAX.TOTL.GD.ZS": "Tax revenue (% of GDP)",
    "GC.XPN.INTP.RV.ZS": "Interest payments (% of revenue)",
    "IC.BUS.EASE.XQ": "Ease of doing business index (1=most business-friendly regulations)",
    "GC.XPN.GSRV.GD.ZS": "Government expenditure on education (% of GDP)",
    "GC.XPN.TOTL.KD.ZG": "Government expenditure growth (annual %)",
    "NE.GDI.FTOT.ZS": "Foreign direct investment, net outflows (% of GDP)",
    "FS.AST.DOMS.GD.ZS": "Domestic credit to private sector (% of GDP)",
    "FS.LBL.MQMY.CD": "Broad money (current LCU)",
    "FS.LBL.MQMY.ZS": "Broad money (% of GDP)",
    "FS.LBL.LIQU.GD.ZS": "Liquid liabilities (% of GDP)",
    "GC.TAX.YPKT.ZS": "Taxes on income, profits, and capital gains (% of revenue)",
    "GC.XPN.INTP.CD": "Interest payments (current US$)",
    "BN.GSR.GNFS.CD": "Net trade in goods and services (BoP, current US$)",
    "NE.GDI.TOTL.KD.ZG": "Gross capital formation (annual % growth)",
    "GC.REV.XGRT.CD": "Revenue, excluding grants (current US$)",
    "NE.EXP.GNFS.CD": "Exports of goods and services (BoP, current US$)",
    "NE.IMP.GNFS.CD": "Imports of goods and services (BoP, current US$)",
    "NY.GDP.TOTL.CD": "Gross Domestic Product, Total (current US$)",
    "FP.CPI.TOTL.ZG": "Inflation, year on year (%)",
    "NY.GDP.PCAP.KD": "GDP per capita (constant 2010 US$)",
    "GC.TAX.TOTL.CD": "Tax revenue (current US$)",
    "GC.XPN.TOTL.CD": "Government total expenditure (current US$)",
    "FI.RES.XGLD.KD": "Total reserves minus gold (constant US$)",
    "FP.CPI.TOTL.KD": "Inflation (constant US$)",
    "NE.GDI.STKB.GD.ZS": "Stock of FDI to GDP (%)",
    "GC.REV.GTOT.ZS": "Government revenue from natural resources (% of GDP)",
    "NY.GDP.TOTL.KD.ZG": "Total GDP growth (annual %)",
    "IC.LGL.CRED.XQ": "Strength of legal rights index (0=weak to 12=strong)",
    "IC.REG.COST.PC": "Cost to register a business (% of GNI per capita)",
    "IC.REG.DURS": "Time required to start a business (days)",
    "IC.BUS.DTF.IX": "Distance to frontier (ease of doing business score)",
    "NE.EXP.MRCH.KD": "Merchandise exports (constant 2010 US$)",
    "NE.IMP.MRCH.KD": "Merchandise imports (constant 2010 US$)",
    "FP.RET.TOTL.KD": "Real interest rate (%)",
    "IC.TAX.LABR.CP.ZS": "Labor tax and contributions (% of commercial profits)",
    "IC.BUS.NREG": "Number of registered businesses",
    "NY.GDP.TOTL.PP.KD": "GDP, PPP (constant international $)",
    "NE.EXP.MRCH.PP.CD": "Merchandise exports, PPP (current international $)",
    "NE.IMP.MRCH.PP.CD": "Merchandise imports, PPP (current international $)",
    "NY.GDP.DEFL.TOTL.ZG": "GDP deflator (annual %)",
    "FS.LBL.MQMY.KD": "Broad money (constant 2010 US$)",
    "FS.LBL.MQMY.PP.CD": "Broad money, PPP (current international $)",
    "NY.GDP.PCAP.KD.ZG": "GDP per capita growth (annual %)",
    "GC.XPN.SERV.CD": "Government expenditure on services (current US$)",
    "NE.EXP.TOTL.KD.ZG": "Exports growth (annual %)",
    "NE.IMP.TOTL.KD.ZG": "Imports growth (annual %)",
    "IC.REG.STRT.CD": "Time to start a business (current US$)",
    "IC.TAX.PRFT.ZS": "Profit tax (% of commercial profits)",
    "IC.TAX.TOTL.ZS": "Total tax rate (% of commercial profits)",
    "NE.EXP.TOTL.ZS": "Exports of goods and services (% of GDP)",
    "NE.IMP.TOTL.ZS": "Imports of goods and services (% of GDP)",
    "EN.ATM.CO2E.PC": "CO2 emissions (metric tons per capita)",
    "EG.FEC.RNEW.ZS": "Renewable energy consumption (% of total final energy consumption)",
    "IT.NET.USER.ZS": "Internet users (per 100 people)",
    "IT.CEL.SETS.P2": "Mobile cellular subscriptions (per 100 people)",
    "AG.LND.TOTL.K2": "Land area (sq. km)",
    "AG.SRF.TOTL.K2": "Surface area (sq. km)",
    "EN.POP.DNST": "Population density (people per sq. km of land area)",
    "IT.USE.RUR.ZS": "Rural population with access to electricity (% of rural population)",
    "IT.USE.URB.ZS": "Urban population with access to electricity (% of urban population)",
    "EN.URB.MCTY.TL.ZS": "Urban population living in slums (% of urban population)",
    "EG.USE.ELEC.KH.PC": "Electricity consumption (kWh per capita)",
    "EN.ATM.CO2E.KD": "CO2 emissions (kilotonnes)",
    "EG.ELC.ACCS.ZS": "Access to electricity (% of population)",
    "EG.FEC.RNEW.ZG": "Growth of renewable energy consumption (annual %)",
    "EG.USE.PCAP.KG.OE": "Energy use (kg of oil equivalent per capita)",
    "EG.USE.COMM.FO.ZS": "Energy use from fossil fuels (% of total)",
    "EG.USE.COMM.CL.ZS": "Energy use from coal (% of total)",
    "EN.ATM.METH.KD": "Methane emissions (kt of CO2 equivalent)",
    "EN.ATM.NOXE.KT": "Nitrous oxide emissions (thousand metric tons of CO2 equivalent)",
    "EN.ATM.PM25.MC.M3": "PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)",
    "EG.USE.COMM.NU.ZS": "Energy use from nuclear sources (% of total)",
    "EG.ELC.ACCS.RU.ZS": "Rural population with access to electricity (% of rural population)",
    "EG.ELC.ACCS.UR.ZS": "Urban population with access to electricity (% of urban population)",
    "IT.MLT.MAIN.P2": "Fixed telephone subscriptions (per 100 people)",
    "EN.POP.DNST.UR": "Urban population density",
    "AG.LND.CREL.HA": "Cereal land (hectares)",
    "AG.LND.FRST.ZS": "Forest area (% of land area)",
    "IT.INT.SERV.ZS": "Households with internet access (% of households)",
    "IT.TEL.SERV.ZS": "Telephone lines (% of households)",
    "EG.FEC.HYDRO.ZS": "Hydropower energy consumption (% of total)",
    "EG.FEC.SOLAR.ZS": "Solar energy consumption (% of total)",
    "EN.ATM.GHGT.ZG": "Greenhouse gas emissions growth (annual %)",
    "IT.NET.BROAD.ZS": "Broadband internet subscriptions (per 100 people)",
    "IT.MOB.PENR.ZS": "Mobile phone penetration rate (% of population)",
    "IT.INT.USER.POP": "Internet users (% of population)",
    # 🌍 Climate change and emissions
    "EN.ATM.GHGT.KT.CE": "Total greenhouse gas emissions (kt of CO2 equivalent)",
    "EN.CLC.MDAT.ZS": "Climate-related natural disaster damages (% of GDP)",
    "EN.CLC.HRSD.ZS": "Population affected by climate shocks (% of population)",
    "EN.ATM.CO2E.LF.KT": "CO2 emissions from liquid fuel consumption (kt)",
    "EN.ATM.CO2E.SF.KT": "CO2 emissions from solid fuel consumption (kt)",
    "EN.ATM.CO2E.GF.KT": "CO2 emissions from gaseous fuel consumption (kt)",
    
    # 🌞 Renewable & alternative energy
    "EG.FEC.GEO.ZS": "Geothermal energy consumption (% of total final energy consumption)",
    "EG.FEC.BIOM.ZS": "Biomass energy use (% of total)",
    "EG.FEC.WIND.ZS": "Wind energy consumption (% of total)",
    "EG.IMP.CONS.ZS": "Electricity imports (% of total electricity use)",
    "EG.IMP.CONS.CD": "Electricity imports (kWh)",

    # 🌿 Environmental protection & forests
    "AG.LND.FRST.K2": "Forest area (sq. km)",
    "AG.LND.ARBL.ZS": "Arable land (% of land area)",
    "AG.LND.IRIG.AG.ZS": "Irrigated land (% of agricultural land)",
    "ER.PTD.TOTL.ZS": "Terrestrial protected areas (% of total land area)",
    "ER.MRN.TOTL.ZS": "Marine protected areas (% of territorial waters)",
    
    # 💧 Water & sanitation infrastructure
    "SH.H2O.SAFE.ZS": "People using safely managed drinking water services (% of population)",
    "SH.STA.BASS.ZS": "People using basic sanitation services (% of population)",
    "ER.H2O.FWTL.ZS": "Freshwater withdrawals (% of internal resources)",
    "SH.STA.HYGN.RU.ZS": "Rural population using basic hygiene services (% of rural population)",
    "SH.STA.HYGN.UR.ZS": "Urban population using basic hygiene services (% of urban population)",

    # ♻️ Waste management
    "EN.WST.PRTM.ZS": "Municipal waste properly treated (% of total waste)",
    "EN.WST.GEN.ZS": "Municipal solid waste generated per capita (kg/day)",
    "EN.WST.COL.ZS": "Waste collection coverage (% of population)",
    
    # 🚧 Transport & logistics
    "IS.ROD.PAVE.ZS": "Roads, paved (% of total roads)",
    "IS.ROD.TOTL.KM": "Road network (total km)",
    "IS.AIR.PSGR": "Air transport, passengers carried",
    "IS.AIR.DPRT": "Air transport, departures worldwide",
    "IS.RRS.TOTL.KM": "Rail lines (total route-km)",
    
    # 🏗️ Infrastructure investment & quality
    "NE.GDI.FITT.CD": "Infrastructure investment, total (current US$)",
    "IS.INV.ROADS.ZS": "Public investment in roads (% of GDP)",
    "IS.INV.ENERGY.ZS": "Public investment in energy (% of GDP)",
    "IS.ELE.PROD.KH": "Electricity production (kWh)",
    "IS.ELE.GEN.REN.ZS": "Electricity from renewables (% of total generation)",
    
    # 🌐 ICT access & usage
    "IT.NET.BBND.CD": "Fixed broadband subscriptions (number)",
    "IT.NET.SECR.ZS": "Population with cybersecurity awareness (% of total)",
    "IT.NET.CMPU.ZS": "Households with a computer (% of households)",
    "IT.TEL.TOTL": "Telephone lines (total)",
    "IT.SAT.USD.CD": "Satellite communication usage (bytes)",
    
    # ⚠️ Disaster & resilience
    "EN.CLC.DSRV.ZS": "Disaster risk reduction policy score (% implemented)",
    "EN.RSK.SHEL.ZS": "Population with access to emergency shelters (% of total)",
    "EN.CLC.RESP.CD": "Public spending on climate resilience (current US$)",
    "EN.CLC.DAMG.CD": "Economic damage from climate disasters (current US$)",
    "EN.RSK.DISAST.ZG": "Disaster risk reduction funding (annual %)",

    # Income inequality & poverty
    "SI.POV.DDAY": "Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)",
    "SI.POV.GINI": "Gini index (World Bank estimate)",

    # External debt & servicing
    "DT.DOD.DECT.CD": "External debt stocks, total (DOD, current US$)",
    "DT.TDS.DECT.EX.ZS": "Total debt service (% of exports of goods, services and primary income)",

    # Terms of trade & balance
    "TT.PRI.MRCH.XD.WD": "Net barter terms of trade index (2000 = 100)",
    "BN.TRF.CURR.CD": "Personal remittances, received (current US$)",
    
    # Employment and wages
    "SL.UEM.TOTL.ZS": "Unemployment, total (% of total labor force) (modeled ILO estimate)",
    "SL.EMP.TOTL.SP.ZS": "Employment to population ratio, 15+, total (%) (modeled ILO estimate)",

    # Productivity
    "NV.IND.MANF.KD.ZG": "Manufacturing, value added (annual % growth)",
    "NV.IND.MANF.ZS": "Manufacturing, value added (% of GDP)",
    "IC.FRM.FEMO.ZS": "Firms with female top manager (% of firms)",
    
    # Investment climate & infrastructure
    "IC.BUS.NREG": "Number of new business registrations (per 1,000 people ages 15-64)",
    "IC.EXP.CSBC": "Time to export, border compliance (hours)",
    "IC.IMP.CSBC": "Time to import, border compliance (hours)",
    "IC.ELC.DURS": "Electricity outages in firms in a typical month (number)",
    
    # Public finance & balance
    "GC.BAL.CASH.GD.ZS": "Cash surplus/deficit (% of GDP)",
    "GC.TAX.GSRV.RV.ZS": "Taxes on goods and services (% of revenue)",
    
    # Commodity dependency
    "TX.VAL.MMTL.ZS.UN": "Ores and metals exports (% of merchandise exports)",
    "TX.VAL.FUEL.ZS.UN": "Fuel exports (% of merchandise exports)",
    "TM.VAL.FUEL.ZS.UN": "Fuel imports (% of merchandise imports)",

    # Capital & financial markets
    "GFDD.DM.06": "Market capitalization of listed domestic companies (% of GDP)",
    "GFDD.OI.02": "Bank nonperforming loans to total gross loans (%)",
    "GFDD.EI.07": "Interest rate spread (lending rate minus deposit rate, %)",

    # Government investment
    "NE.GDI.GOVT.ZS": "Gross fixed capital formation by general government (% of GDP)",

    # Current account & reserves, more detail
    "BN.CUR.CD": "Current account balance (BoP, current US$)",

    # Broader household indicators
    "NE.CON.PRVT.CD": "Household final consumption expenditure (current US$)",
    "NE.CON.GOVT.CD": "Government final consumption expenditure (current US$)",
    "NY.GDP.MKTP.KD": "GDP (constant 2010 US$)"
}


demographic_indicators = {
    "SP.POP.TOTL": "Population, total",
    "SP.DYN.LE00.IN": "Life expectancy at birth, total (years)",
    "SI.POV.DDAY": "Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)",
    "SE.ADT.LITR.ZS": "Literacy rate, adult total (% of people ages 15 and above)",
    "SH.DYN.MORT": "Mortality rate, under-5 (per 1,000 live births)",
    "SH.STA.MMRT": "Maternal mortality ratio (modeled estimate, per 100,000 live births)",
    "SP.RUR.TOTL": "Rural population",
    "SP.URB.TOTL": "Urban population",
    "SE.XPD.TOTL.GD.ZS": "Government expenditure on education, total (% of GDP)",
    "SP.DYN.TFRT.IN": "Fertility rate, total (births per woman)",
    "SP.POP.GROW": "Population growth (annual %)",
    "SP.DYN.IMRT.IN": "Mortality rate, infant (per 1,000 live births)",
    "SP.DYN.TO65.FE.ZS": "Population ages 65 and above, female (% of female population)",
    "SP.DYN.TO65.MA.ZS": "Population ages 65 and above, male (% of male population)",
    "SP.DYN.CDRT.FE": "Death rate, crude (per 1,000 female adults)",
    "SP.DYN.CDRT.MA": "Death rate, crude (per 1,000 male adults)",
    "SP.DYN.LE00.MA.IN": "Life expectancy at birth, male (years)",
    "SP.DYN.LE00.FE.IN": "Life expectancy at birth, female (years)",
    "SP.POP.1564.TO": "Population ages 15-64, total",
    "SP.POP.65UP.TO": "Population ages 65 and above, total",
    "SP.POP.0014.TO": "Population ages 0-14, total",
    "SP.POP.65UP.MA.TO": "Population ages 65 and above, male",
    "SP.POP.65UP.FE.TO": "Population ages 65 and above, female",
    "SP.URB.GROW": "Urban population growth (annual %)",
    "SP.POP.TOTL.FE.ZS": "Female population (% of total population)",
    "SP.POP.TOTL.MA.ZS": "Male population (% of total population)",
    "SP.POP.SCIE.RD": "Researchers in R&D (per million people)",
    "SP.POP.0014.FE.ZS": "Female population ages 0-14 (% of total female population)",
    "SP.POP.0014.MA.ZS": "Male population ages 0-14 (% of total male population)",
    "SP.POP.SCIE.ZS": "Researchers in R&D (% of population)",
    "SP.DYN.AMRT.MA": "Adult mortality rate, male (per 1,000 male adults)",
    "SP.DYN.AMRT.FE": "Adult mortality rate, female (per 1,000 female adults)",
    "SP.DYN.MRRT.MA": "Maternal mortality rate (per 100,000 live births)",
    "SP.DYN.MRRT.FE": "Maternal mortality rate (female)",
    "SP.HOU.FAML.FE.ZS": "Female-headed households (% of households)",
    "SP.HOU.FAML.MA.ZS": "Male-headed households (% of households)",
    "SP.POP.TOTL.UR.ZS": "Urban population (% of total)",
    "SP.POP.TOTL.RU.ZS": "Rural population (% of total)",
    "SP.RUR.TOTL.ZG": "Rural population growth (annual %)",
    "SP.POP.SCIE.MA.ZS": "Male researchers in R&D (% of male population)",
    "SP.POP.SCIE.FE.ZS": "Female researchers in R&D (% of female population)",
    "SP.DYN.TO10.FE.ZS": "Population ages 0-10, female (% of female population)",
    "SP.DYN.TO10.MA.ZS": "Population ages 0-10, male (% of male population)",
    "SP.DYN.TFRT.UR.ZS": "Urban fertility rate (births per woman)",
    "SP.DYN.TFRT.RU.ZS": "Rural fertility rate (births per woman)",
    "SP.DYN.CDRT.UR.ZS": "Urban crude death rate (per 1,000 population)",
    "SP.DYN.CDRT.RU.ZS": "Rural crude death rate (per 1,000 population)",
    "SP.POP.TOTL.FG": "Foreign-born population (% of total population)",
    "SP.POP.TOTL.MI": "Migrant population (% of total population)",
    "SP.POP.GROW.0014": "Population growth (ages 0-14)",
    "SP.POP.GROW.1564": "Population growth (ages 15-64)",
    "SP.POP.GROW.65UP": "Population growth (ages 65 and above)",
    "SP.POP.SCI.TOTL": "Total population in science and engineering",
    "SP.POP.TECH.TOTL": "Total population in technology",
    "SP.POP.EDUC.TOTL": "Total population with tertiary education",
    "SP.POP.FRTT.TOTL": "Fertility rate, teenage (births per 1,000 women ages 15-19)",
    "SP.POP.RUR.CB": "Rural population (% of total population)",
    "SP.POP.URB.CB": "Urban population (% of total population)",
    "SP.POP.MARR.TOTL": "Married population (total)",
    # Migration & diaspora
    "SM.POP.NETM": "Net migration",
    "SM.POP.REFG": "Refugee population by country or territory of asylum",
    "SM.POP.REFI": "Refugee population by country of origin",
    "SM.POP.TOTL.ZS": "Migrant population (% of total population)",
    "BX.TRF.PWKR.DT.GD.ZS": "Personal remittances, received (% of GDP)",

    # Health-related demographic indicators
    "SH.MED.BEDS.ZS": "Hospital beds (per 1,000 people)",
    "SH.STA.HYGN.ZS": "People using at least basic sanitation services (% of population)",
    "SH.STA.WASH.ZS": "People using at least basic drinking water services (% of population)",
    "SH.TBS.INCD": "Incidence of tuberculosis (per 100,000 people)",
    "SH.IMM.MEAS": "Immunization, measles (% of children ages 12-23 months)",

    # Age dependency & demographic pressure
    "SP.POP.DPND": "Age dependency ratio (% of working-age population)",
    "SP.POP.DPND.YG": "Age dependency ratio, young (% of working-age population)",
    "SP.POP.DPND.OL": "Age dependency ratio, old (% of working-age population)",
    "SP.POP.80UP.TO": "Population ages 80 and above, total",
    "SP.POP.80UP.FE.ZS": "Population ages 80+, female (% of female population)",
    "SP.POP.80UP.MA.ZS": "Population ages 80+, male (% of male population)",

    # Births, deaths, and growth projections
    "SP.REG.BRTH.ZS": "Birth registration coverage (% of children under age 5)",
    "SP.REG.DEAD.ZS": "Death registration coverage (% of deceased over age 5)",
    "SP.DYN.BRTH.IN": "Crude birth rate (per 1,000 people)",
    "SP.DYN.DEATH.IN": "Crude death rate (per 1,000 people)",
    "SP.POP.TOTL.PROJ": "Projected population (2030)",
    "SP.POP.1564.PROJ": "Projected working-age population (15-64, 2030)",

    # Education-related demographics
    "SE.PRM.NENR": "Adjusted net enrollment rate, primary (% of primary school-age children)",
    "SE.SEC.NENR": "Adjusted net enrollment rate, secondary (% of secondary school-age children)",
    "SE.TER.ENRR": "Gross enrollment ratio, tertiary (%)",
    "SE.ADT.LITR.MA.ZS": "Literacy rate, adult male (% of males ages 15 and above)",
    "SE.ADT.LITR.FE.ZS": "Literacy rate, adult female (% of females ages 15 and above)",
    "SE.ENR.PRSC.FM.ZS": "Ratio of female to male enrollment in primary and secondary education (%)",

    # Gender-specific & vulnerable group statistics
    "SG.VAW.1549.ZS": "Women subjected to violence by intimate partner (% of women ages 15-49)",
    "SG.GEN.PARL.ZS": "Proportion of seats held by women in national parliaments (%)",
    "SG.LAW.INDX": "Women, business and the law index (scale 0–100)",
    "SP.MTR.1519.ZS": "Adolescent fertility rate (births per 1,000 women ages 15-19)",
    "SG.VAW.REAS.ZS": "Reported reasons for not reporting violence (as % of victims)",

    # Additional urban & rural breakdowns
    "EN.URB.MCTY.TL.ZS": "Urban population living in slums (% of urban population)",
    "SP.URB.TOTL.IN.ZS": "Urban population (% of total, interpolated)",
    "SP.RUR.TOTL.IN.ZS": "Rural population (% of total, interpolated)",
    "SP.URB.BRTH.ZS": "Urban birth rate (per 1,000 population)",
    "SP.RUR.BRTH.ZS": "Rural birth rate (per 1,000 population)",

    # Family structure & household statistics
    "SP.HOU.TOTL": "Number of households",
    "SP.HOU.AVGSIZE": "Average household size",
    "SP.HOU.SINGLE.ZS": "Single-person households (% of all households)",

    # Disability & social protection
    "SP.DIS.SOCU.ZS": "Population with disability receiving social protection (% of total)",
    "SP.DIS.FEMP.ZS": "Female disabled population (% of total disabled)",
    "SP.DIS.CHIL.ZS": "Children with disability (% of all children)",

    # Labor force demographic features (useful in overlap with econ)
    "SL.TLF.TOTL.IN": "Total labor force",
    "SL.TLF.TOTL.FE.ZS": "Labor force participation rate, female (% of female population ages 15+)",
    "SL.TLF.TOTL.MA.ZS": "Labor force participation rate, male (% of male population ages 15+)",
    "SL.TLF.CACT.FM.ZS": "Ratio of female to male labor force participation rate (%)",
    "SP.POP.UNEMP.TOTL": "Unemployed population (total)"
}



political_indicators = {
    "GC.REV.XGRT.GD.ZS": "Revenue, excluding grants (% of GDP)",
    "GC.XPN.TOTL.GD.ZS": "Expense (% of GDP)",
    "GC.NLD.TOTL.GD.ZS": "Net lending (+) / net borrowing (-) (% of GDP)",
    "GC.TAX.TOTL.GD.ZS": "Tax revenue (% of GDP)",
    "IC.BUS.EASE.XQ": "Ease of doing business index (1=most business-friendly regulations)",
    "SI.POV.GINI": "Gini index (World Bank estimate)",
    "IC.LGL.CRED.XQ": "Strength of legal rights index (0=weak to 12=strong)",
    "IC.BUS.NREG": "Number of registered businesses",
    "IC.LGL.PROP.XQ": "Property rights index (0=weak to 100=strong)",
    "IC.REG.COST.PC": "Cost to register a business (% of GNI per capita)",
    "IC.REG.DURS": "Time required to start a business (days)",
    "IC.REG.PROP.XQ": "Property registration index (0=weak to 12=strong)",
    "IC.REG.STRT.DUR": "Time to start a business (days)",
    "IC.TAX.LABR.CP.ZS": "Labor tax and contributions (% of commercial profits)",
    "IC.TAX.GDGT.ZS": "Gross debt to GDP ratio (%)",
    "IC.TAX.PRFT.ZS": "Profit tax (% of commercial profits)",
    "IC.TAX.TOTL.ZS": "Total tax rate (% of commercial profits)",
    "IC.TAX.TOTL.CD": "Total tax rate (current LCU)",
    "IC.TAX.GOODS.ZS": "Goods and services tax (% of value added)",
    "IC.TAX.TOTL.KD": "Total tax rate (constant LCU)",
    "IC.TAX.GOODS.CD": "Goods and services tax (current LCU)",
    "IC.REG.DURS.KD": "Time required to start a business (constant LCU)",
    "IC.REG.DURS.CD": "Time required to start a business (current LCU)",
    "IC.REG.PROP.KD": "Property registration index (constant LCU)",
    "IC.REG.PROP.CD": "Property registration index (current LCU)",
    "IC.REG.STRT.KD": "Time to start a business (constant LCU)",
    "IC.REG.STRT.CD": "Time to start a business (current LCU)",
    "IC.LGL.CRED.KD": "Strength of legal rights index (constant LCU)",
    "IC.LGL.CRED.CD": "Strength of legal rights index (current LCU)",
    "IC.LGL.PROP.KD": "Property rights index (constant LCU)",
    "IC.LGL.PROP.CD": "Property rights index (current LCU)",
    "IC.LGL.PROP.ZG": "Property rights index (% change from previous year)",
    "IC.LGL.CRED.ZG": "Strength of legal rights index (% change from previous year)",
    "IC.REG.COST.ZG": "Cost to register a business (% change from previous year)",
    "IC.TAX.LABR.ZG": "Labor tax and contributions (% change from previous year)",
    "IC.TAX.GDGT.ZG": "Gross debt to GDP ratio (% change from previous year)",
    "IC.TAX.PRFT.ZG": "Profit tax (% change from previous year)",
    "IC.TAX.TOTL.ZG": "Total tax rate (% change from previous year)",
    "IC.TAX.GOODS.ZG": "Goods and services tax (% change from previous year)",
    "IC.REG.DURS.ZG": "Time required to start a business (% change from previous year)",
    "IC.REG.PROP.ZG": "Property registration index (% change from previous year)",
    "IC.REG.STRT.ZG": "Time to start a business (% change from previous year)",
    "IC.REG.DURS.ZG.KD": "Time required to start a business (% change from previous year, constant LCU)",
    "IC.REG.DURS.ZG.CD": "Time required to start a business (% change from previous year, current LCU)",
    "IC.REG.PROP.ZG.KD": "Property registration index (% change from previous year, constant LCU)",
    "IC.REG.PROP.ZG.CD": "Property registration index (% change from previous year, current LCU)",
    "IC.REG.STRT.ZG.KD": "Time to start a business (% change from previous year, constant LCU)",
    "IC.REG.STRT.ZG.CD": "Time to start a business (% change from previous year, current LCU)",
    "IC.LGL.PROP.ZS": "Property rights index (% of maximum)",
    "IC.LGL.CRED.ZS": "Legal rights index (% of maximum)",
    "GC.REV.GTOT.CD": "Government revenue, total (current US$)",
    "GC.XPN.TOTL.GD.ZG": "Government expenditure growth (annual %)",
    "GC.REV.XGRT.CD": "Revenue excluding grants (current US$)",
    "GC.TAX.YPKT.CD": "Taxes on income, profits and capital gains (current US$)",
    "GC.XPN.INTP.GD.ZS": "Interest payments (% of GDP)",
    "IC.LGL.PROP.XY": "Property rights index (0=weak to 100=strong)",
    "GC.XPN.GSRV.GD.ZS": "Expenditure on general public services (% of GDP)",
    "GC.REV.XGRT.KD.ZG": "Revenue excluding grants (annual % growth)",
    "IC.BUS.REG.BD.ZS": "Cost of business regulation (% of GNI per capita)",
    "IC.REG.BD.YR": "Years to enforce a contract (years)",
    "GC.TAX.YPKT.ZG": "Income tax growth (annual %)",
    "IC.REG.ENF.TOTL": "Contract enforcement time (days)",
    "IC.BUS.DTF.EASE": "Ease of doing business score",
    "GC.REV.GTOT.GD.ZS": "Total government revenue (% of GDP)",
    "GC.XPN.GSRV.KD.ZG": "Public services expenditure growth (annual %)",
    "GC.REV.TAX.KD.ZG": "Tax revenue growth (annual %)",
    "IC.BUS.LAB.MKTS": "Labor market efficiency index",
    "IC.LGL.JST.XQ": "Judicial effectiveness index (0=weak to 100=strong)",
    "GC.REV.PUB.GOVT": "Government revenues (general government)",
     # Governance indicators from World Governance Indicators (WGI)
    "VA.EST": "Voice and Accountability (Estimate)",
    "VA.PER": "Voice and Accountability (Percentile Rank)",
    "VA.SES": "Voice and Accountability (Standard Error)",

    "PV.EST": "Political Stability and Absence of Violence/Terrorism (Estimate)",
    "PV.PER": "Political Stability and Absence of Violence/Terrorism (Percentile Rank)",
    "PV.SES": "Political Stability and Absence of Violence/Terrorism (Standard Error)",

    "GE.EST": "Government Effectiveness (Estimate)",
    "GE.PER": "Government Effectiveness (Percentile Rank)",
    "GE.SES": "Government Effectiveness (Standard Error)",

    "RQ.EST": "Regulatory Quality (Estimate)",
    "RQ.PER": "Regulatory Quality (Percentile Rank)",
    "RQ.SES": "Regulatory Quality (Standard Error)",

    "RL.EST": "Rule of Law (Estimate)",
    "RL.PER": "Rule of Law (Percentile Rank)",
    "RL.SES": "Rule of Law (Standard Error)",

    "CC.EST": "Control of Corruption (Estimate)",
    "CC.PER": "Control of Corruption (Percentile Rank)",
    "CC.SES": "Control of Corruption (Standard Error)",

    # Transparency & corruption
    "IC.TRAN.PUB.ZS": "Public sector transparency index (% of transparency achieved)",
    "IC.TRAN.CORR.ZS": "Corruption perception index (0=high corruption to 100=no corruption)",

    # Decentralization
    "GC.XPN.SUBN.CD": "Subnational government expenditure (current US$)",
    "GC.REV.SUBN.CD": "Subnational government revenue (current US$)",
    "GC.REV.SUBN.GD.ZS": "Subnational revenue (% of GDP)",
    "GC.XPN.SUBN.GD.ZS": "Subnational expenditure (% of GDP)",

    # Elections and civic participation (if available)
    "SG.ELE.TURN.ZS": "Voter turnout (% of registered voters)",
    "SG.ELE.FAIR.ZS": "Electoral process and pluralism index (0-100)",
    "SG.CIV.LIB.ZS": "Civil liberties index (0-100)",

    # Government structure & capacity
    "GC.GOV.EMPL.TOTL": "Total public sector employees",
    "GC.GOV.WAGE.GD.ZS": "Wages and salaries in general government (% of GDP)",
    "GC.REV.NTRN.GD.ZS": "Non-tax revenue (% of GDP)",

    # Rule of law & justice
    "IC.LGL.CASE.TOTL": "Number of court cases resolved annually",
    "IC.LGL.JUDI.INDEX": "Judicial independence index (0=low to 100=high)",
    "IC.LGL.DELAY.DAYS": "Average length of legal proceedings (days)",

    # Contract & property rights enforcement
    "IC.REG.ENF.DUR": "Time to enforce contracts (days)",
    "IC.REG.ENF.COST": "Cost to enforce contracts (% of claim value)",
    "IC.LGL.PROP.ENF": "Property rights enforcement index (0–100)",

    # Budget transparency & participatory governance
    "GC.BUD.TSCP.ZS": "Budget transparency score (% of maximum)",
    "SG.PAR.INCL.ZS": "Participatory governance index (% of participation mechanisms fulfilled)",
    
    # Additional institutional indicators
    "SG.POL.CIVI.ZS": "Civic engagement index",
    "SG.POL.INDX": "Political rights index (0=worst, 100=best)",
    "SG.POL.ACCS.ZS": "Access to government information (% of population)",
    "SG.PUB.ACCS.ZS": "Public accountability mechanisms index",
    
    # Executive & legislative capacity
    "SG.EXE.CAPA.ZS": "Executive capacity index (0-100)",
    "SG.LEG.CAPA.ZS": "Legislative oversight index (0-100)",
    "GC.REV.LOC.GOVT": "Revenue, local governments (US$)"
}



security_indicators = {
    "MS.MIL.XPND.GD.ZS": "Military expenditure (% of GDP)",
    "SL.UEM.TOTL.ZS": "Unemployment, total (% of total labor force) (modeled ILO estimate)",
    "SL.UEM.TOTL.NE.ZS": "Unemployment, total (% of total labor force) (national estimate)",
    "MS.MIL.XPND.CD": "Military expenditure (current US$)",
    "MS.MIL.XPND.KD": "Military expenditure (constant US$)",
    "MS.MIL.TOTL.PP": "Military personnel (per 100,000 people)",
    "MS.MIL.TOTL": "Armed forces personnel, total",
    "MS.MIL.RATIO": "Armed forces personnel (% of total labor force)",
    "MS.TROOPS.DPLD": "Troops deployed abroad (number)",
    "MS.TROOPS.ACTV": "Active military personnel",
    "MS.MIL.AID.US": "Military aid received (current US$)",
    "SL.TLF.ACTV": "Labor force participation rate (female, male)",
    "MS.SEC.AID.PUB": "Public security aid received (current US$)",
    "SL.UEM.1524.MA.ZS": "Unemployment, youth male (% of male labor force ages 15-24)",
    "SL.UEM.1524.FE.ZS": "Unemployment, youth female (% of female labor force ages 15-24)",
    "SL.UEM.TOTL.FE.ZS": "Unemployment, female (% of female labor force)",
    "SL.UEM.TOTL.MA.ZS": "Unemployment, male (% of male labor force)",
    "SL.EMP.VULN.FE.ZS": "Vulnerable employment, female (% of female employment)",
    "SL.EMP.VULN.MA.ZS": "Vulnerable employment, male (% of male employment)",
    "SL.EMP.WORK.MA.ZS": "Wage and salaried workers, male (% of male employment)",
    "SL.EMP.WORK.FE.ZS": "Wage and salaried workers, female (% of female employment)",
    "SL.EMP.TOTL.SP.ZS": "Employment to population ratio, total (%) (modeled ILO estimate)",
    "SL.EMP.SELF.FE.ZS": "Self-employed, female (% of female employment)",
    "SL.EMP.SELF.MA.ZS": "Self-employed, male (% of male employment)",
    "SL.EMP.SELF.ZS": "Self-employed, total (% of total employment)",
    "SL.EMP.SELF.TOTL": "Self-employed, total",
    "SL.UEM.1524.ZS": "Unemployment, youth total (% of labor force ages 15-24)",
    "MS.MIL.TOTL.TROOP": "Military troops, total",
    "SL.UEM.NEET.ZS": "Youth not in education, employment or training (% of youth population)",
    "MS.MIL.TOTL.ACT": "Active military troops, total",
    "MS.MIL.TOTL.RES": "Reserve military troops, total",
    "MS.MIL.TOTL.STAFF": "Military personnel, staff officers",
    "SL.EMP.TOTL.ACT.M": "Active labor force, male",
    "SL.EMP.TOTL.ACT.F": "Active labor force, female",
      # Policing and law enforcement
    "SG.LAW.POLI.FE": "Police personnel, female (per 100,000 population)",
    "SG.LAW.POLI.MA": "Police personnel, male (per 100,000 population)",
    "SG.LAW.POLI.TOTL": "Police personnel (per 100,000 population)",
    "SG.LAW.COURT.ZS": "Judicial system capacity (% of cases resolved)",

    # Fragility, conflict & violence
    "VC.IHR.PSRC.P5": "Intentional homicides (per 100,000 people)",
    "VC.BTL.DETH": "Battle-related deaths (number of people)",
    "VC.VAW.RAPE.ZS": "Reported cases of rape (per 100,000 people)",
    "VC.TRF.HMN.ZS": "Human trafficking victims (% of population)",
    "VC.DRM.DIS": "Number of deaths from disasters (per year)",
    "VC.FRG.FS": "Fragile States Index (score 0–120)",
    "VC.VIO.CONF.ZS": "Population exposed to conflict-related violence (% of total)",
    
    # Peacekeeping & military alliances
    "MS.PEAC.KPR.TOTL": "Peacekeeping personnel contributed (number)",
    "MS.PEAC.KPR.GDP.ZS": "Peacekeeping expenditure (% of GDP)",
    "MS.ALLI.BDGT": "Defense alliance budget contribution (current US$)",
    
    # Arms and equipment
    "MS.ARM.IMP.CD": "Arms imports (SIPRI trend indicator values, current US$)",
    "MS.ARM.EXP.CD": "Arms exports (SIPRI trend indicator values, current US$)",
    "MS.ARM.EXP.GD.ZS": "Arms exports (% of GDP)",
    "MS.ARM.IMP.GD.ZS": "Arms imports (% of GDP)",

    # Security expenditure breakdowns
    "MS.SEC.PUB.XPND.CD": "Public security and safety expenditure (current US$)",
    "MS.SEC.PUB.XPND.GD.ZS": "Public security and safety expenditure (% of GDP)",
    "MS.MIL.XPND.GD.ZG": "Military expenditure growth (annual %)",
    "MS.MIL.XPND.PP": "Military expenditure per capita (US$)",
    
    # Gender and security
    "MS.MIL.FEM.PERS.ZS": "Female military personnel (% of total armed forces)",
    "MS.MIL.MALE.PERS.ZS": "Male military personnel (% of total armed forces)",
    
    # Defense budget efficiency
    "MS.MIL.XPND.EFF.ZS": "Military spending efficiency index (% of effectiveness)",
    "MS.MIL.EQUIP.CAP": "Military equipment capacity index (0–100)",

    # Security-related labor force fragility
    "SL.UEM.LONG.ZS": "Long-term unemployment (% of total unemployment)",
    "SL.UEM.INAC.ZS": "Inactive youth not in labor force (% of youth population)",
    "SL.EMP.UNDER.ZS": "Underemployment (% of total labor force)",
    
    # Resilience & recovery
    "VC.RSK.RES.ZS": "Population with access to basic disaster risk resilience (% of total)",
    "VC.DRM.RESP.ZS": "Government emergency response index (0–100)",
    
    # National service & reserve systems
    "MS.MIL.NAT.SERV.ZS": "Population enrolled in national service (% of total labor force)",
    "MS.MIL.RES.CAP.ZS": "Reserve forces capacity (% of standing army)",

    # Cyber & strategic security (if available)
    "MS.SEC.CYBR.IDX": "Cybersecurity capability index (0–100)",
    "MS.STR.THRT.IDX": "Strategic threat index (0–100)",
    "SL.EMP.VULN.YOUTH.ZS": "Vulnerable youth employment, total (% of total employment)"
}

# Define the date range
#start_date = "1960-01-01"
#end_date = "2024-01-01"

############################
# Period of interest
#########################
start_year = 1960
end_year = 2024


## Data Collection

In [5]:
# --- Pre-reqs: make sure these exist somewhere above ---
# African_country_groups: dict like {"ECOWAS": ["CIV","GHA",...], ...}
# fetch_data(indicators: list[str] | dict, countries: list[str]) -> dict[str, pd.DataFrame] | pd.DataFrame
# economic_indicators, demographic_indicators, political_indicators, security_indicators

# ----------------- FIXED & HARDENED VERSION -----------------

# 1) Initialize result containers to avoid NameError and to keep a consistent structure.
data_economic: dict = {}
data_demographic: dict = {}
data_political: dict = {}
data_security: dict = {}

# 2) Bundle categories to avoid repeating near-identical code.
categories = [
    ("Economic", "📊", economic_indicators, data_economic),
    ("Demographic", "👨‍👩‍👧‍👦", demographic_indicators, data_demographic),
    ("Political", "🗳️", political_indicators, data_political),
    ("Security", "🛡️", security_indicators, data_security),
]

# 3) Fetch data for each region and category with basic error handling.
for region_code, countries in African_country_groups.items():
    print(f"\n🔄 Fetching data for region: {region_code}")
    for label, emoji, indicators, bucket in categories:
        try:
            # fetch_data should return either:
            #   - dict[str, DataFrame] (e.g., keyed by indicator), OR
            #   - a single DataFrame
            bucket[region_code] = fetch_data(indicators, countries)
            print(f"  ✅ {label} fetched.")
        except Exception as e:
            # On failure, record an empty dict to keep shape consistent and continue.
            bucket[region_code] = {}
            print(f"  ⚠️ {label} fetch failed for {region_code}: {e}")

# 4) Helper to safely print the first rows for any category/region result.
def _print_first_rows(title_emoji: str, label: str, bucket: dict, region: str, n: int = 5):
    """Safely print the head() of the first DataFrame found for a region/category."""
    print(f"\n{title_emoji} {label} Data:")
    region_payload = bucket.get(region)

    if region_payload is None or region_payload == {}:
        print("No data")
        return

    try:
        # Handle both shapes:
        # a) dict[str, DataFrame] → pick first DF
        # b) DataFrame → print directly
        if isinstance(region_payload, dict):
            if not region_payload:
                print("No data")
                return
            first_key = next(iter(region_payload))
            df = region_payload[first_key]
        else:
            df = region_payload  # assume it's already a DataFrame

        # Try to print its head
        print(df.head(n))
    except Exception as e:
        print(f"Could not display data preview: {e}")

# 5) Display a sample preview per region/category.
for region_code in African_country_groups:
    print(f"\n=== Region: {region_code} ===")
    _print_first_rows("📊", "Economic",   data_economic,   region_code)
    _print_first_rows("👨‍👩‍👧‍👦", "Demographic", data_demographic, region_code)
    _print_first_rows("🗳️", "Political",  data_political,  region_code)
    _print_first_rows("🛡️", "Security",   data_security,   region_code)

# 6) Done.
print("\n✅ Complete fetching done!")



🔄 Fetching data for region: NA
🔎 Fetching: GDP (current US$) (NY.GDP.MKTP.CD)
✅ Success: GDP (current US$)
🔎 Fetching: GDP per capita (current US$) (NY.GDP.PCAP.CD)
✅ Success: GDP per capita (current US$)
🔎 Fetching: GDP growth (annual %) (NY.GDP.MKTP.KD.ZG)
✅ Success: GDP growth (annual %)
🔎 Fetching: Inflation, consumer prices (annual %) (FP.CPI.TOTL)
✅ Success: Inflation, consumer prices (annual %)
🔎 Fetching: Exports of goods and services (% of GDP) (NE.EXP.GNFS.ZS)
✅ Success: Exports of goods and services (% of GDP)
🔎 Fetching: Imports of goods and services (% of GDP) (NE.IMP.GNFS.ZS)
✅ Success: Imports of goods and services (% of GDP)
🔎 Fetching: Current account balance (% of GDP) (BN.CAB.XOKA.GD.ZS)
✅ Success: Current account balance (% of GDP)
🔎 Fetching: Foreign direct investment, net inflows (% of GDP) (BX.KLT.DINV.WD.GD.ZS)
✅ Success: Foreign direct investment, net inflows (% of GDP)
🔎 Fetching: Gross savings (% of GDP) (NY.GNS.ICTR.ZS)
✅ Success: Gross savings (% of GDP)
🔎

## Merging Table

In [6]:
import pandas as pd

# --- CONFIG: common column name variants you might have from different sources ---
COUNTRY_COLS  = ["country", "Country", "Country Name", "country_name", "ADMIN", "COUNTRY"]
ISO3_COLS     = ["iso3", "ISO3", "countryiso3code", "Country Code", "adm0_a3", "ISO_A3"]
VALUE_COLS    = ["value", "Value", "OBS_VALUE", "obs_value", "Indicator Value", "val"]
YEAR_COLS     = ["year", "Year", "date", "Date", "time", "TIME", "period"]

def _pick_col(df: pd.DataFrame, candidates: list[str]) -> str | None:
    """Return the first column name that exists in df from a candidate list."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _normalize_one_df(df: pd.DataFrame,
                      indicator: str,
                      region_code: str,
                      category_name: str) -> pd.DataFrame:
    """
    Standardize a single indicator DataFrame to a minimal schema:
    country, variable, value, (optional iso, year), plus region & category context.
    """
    if df is None or getattr(df, "empty", False):
        return pd.DataFrame(columns=["country", "variable", "value", "iso", "year", "region", "category"])

    # Always work on a copy to avoid side effects
    d = df.copy()

    # Reset index if needed
    if isinstance(d.index, pd.MultiIndex) or d.index.name is not None:
        d = d.reset_index()

    # Identify key columns
    ctry = _pick_col(d, COUNTRY_COLS)
    iso  = _pick_col(d, ISO3_COLS)
    val  = _pick_col(d, VALUE_COLS)
    yr   = _pick_col(d, YEAR_COLS)

    # If we truly can't find a value column, try a best-effort guess:
    if val is None:
        # Heuristic: take the first numeric column that isn't obviously an id/name
        numeric_candidates = [c for c in d.columns if pd.api.types.is_numeric_dtype(d[c])]
        drop_these = set((ctry or "", iso or "", yr or "")).union({"index", "gid", "id", "ID"})
        numeric_candidates = [c for c in numeric_candidates if c not in drop_these]
        val = numeric_candidates[0] if numeric_candidates else None

    # Minimal validation: we need a country and a value to produce a usable row
    if ctry is None or val is None:
        # Return empty but well-formed frame to allow safe concat
        return pd.DataFrame(columns=["country", "variable", "value", "iso", "year", "region", "category"])

    # Build standardized frame
    out = pd.DataFrame({
        "country": d[ctry],
        "variable": indicator,        # <- variable name comes from the indicator key
        "value": pd.to_numeric(d[val], errors="coerce"),
        "region": region_code,
        "category": category_name
    })

    # Optional columns if available
    if iso is not None:
        out["iso"] = d[iso].astype(str)

    if yr is not None:
        # Normalize year to YYYY if a full date is present
        yr_series = d[yr].astype(str).str.extract(r"(\d{4})")[0]
        out["year"] = pd.to_numeric(yr_series, errors="coerce").astype("Int64")

    # Drop rows without a country or value
    out = out.dropna(subset=["country", "value"])

    return out

def _flatten_category(regional_store: dict, category_name: str) -> pd.DataFrame:
    """
    Flattens a nested dict:
      {region_code: {indicator_code: DataFrame}}  -> tidy long DataFrame
    with standardized columns.
    """
    frames = []
    for region_code, indicators in regional_store.items():
        if not indicators:
            continue
        for indicator_code, df in indicators.items():
            frames.append(_normalize_one_df(df, indicator_code, region_code, category_name))
    # Safe concat (even if some are empty)
    return pd.concat(frames, ignore_index=True, sort=False) if frames else pd.DataFrame(
        columns=["country", "variable", "value", "iso", "year", "region", "category"]
    )

# --- BUILD: one long table from your already-collected stores ---
economic_long    = _flatten_category(data_economic,    "Economic")
demographic_long = _flatten_category(data_demographic, "Demographic")
political_long   = _flatten_category(data_political,   "Political")
security_long    = _flatten_category(data_security,    "Security")

merged_long = pd.concat(
    [economic_long, demographic_long, political_long, security_long],
    ignore_index=True, sort=False
)

# Optional: tidy up whitespace in country names
merged_long["country"] = merged_long["country"].astype(str).str.strip()

# Optional: order columns for readability
preferred_order = ["country", "iso", "year", "variable", "value", "region", "category"]
merged_long = merged_long[[c for c in preferred_order if c in merged_long.columns] +
                          [c for c in merged_long.columns if c not in preferred_order]]

# --- PREVIEW / USE ---
print("\n📄 Merged long table (head):")
print(merged_long.head())

# Example: filter or sort if needed
# merged_long.query("country == \"Côte d'Ivoire\"").sort_values(["year", "variable"]).head()

# Save if you want
# merged_long.to_parquet("africa_all_variables_long.parquet", index=False)
# merged_long.to_csv("africa_all_variables_long.csv", index=False)


/tmp/ipykernel_3385116/1763280577.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(frames, ignore_index=True, sort=False) if frames else pd.DataFrame(
/tmp/ipykernel_3385116/1763280577.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(frames, ignore_index=True, sort=False) if frames else pd.DataFrame(
/tmp/ipykernel_3385116/1763280577.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future vers


📄 Merged long table (head):
   country  iso  year        variable         value region  category
0  Algeria  NaN  2024  NY.GDP.MKTP.CD  2.636198e+11     NA  Economic
1  Algeria  NaN  2023  NY.GDP.MKTP.CD  2.476262e+11     NA  Economic
2  Algeria  NaN  2022  NY.GDP.MKTP.CD  2.256385e+11     NA  Economic
3  Algeria  NaN  2021  NY.GDP.MKTP.CD  1.862312e+11     NA  Economic
4  Algeria  NaN  2020  NY.GDP.MKTP.CD  1.648734e+11     NA  Economic


## ISO MAPPING

In [7]:
import pandas as pd

# 1) Your manual mapping (as given earlier)
name_to_iso = {
    'Algeria':'DZA','Angola':'AGO','Benin':'BEN','Burkina Faso':'BFA','Burundi':'BDI',
    'Cabo Verde':'CPV','Cape Verde':'CPV','Cameroon':'CMR','Central African Republic':'CAF',
    'Chad':'TCD','Comoros':'COM','Congo, Dem. Rep.':'COD','Congo, Rep.':'COG',"Cote d'Ivoire":'CIV',
    'Djibouti':'DJI','Egypt, Arab Rep.':'EGY','Equatorial Guinea':'GNQ','Eritrea':'ERI','Ethiopia':'ETH',
    'Gabon':'GAB','Gambia, The':'GMB','Ghana':'GHA','Guinea':'GIN','Guinea-Bissau':'GNB','Kenya':'KEN',
    'Liberia':'LBR','Libya':'LBY','Madagascar':'MDG','Malawi':'MWI','Mali':'MLI','Mauritania':'MRT',
    'Morocco':'MAR','Niger':'NER','Nigeria':'NGA','Rwanda':'RWA','Sao Tome and Principe':'STP',
    'São Tomé and Principe':'STP','Senegal':'SEN','Sierra Leone':'SLE','Somalia':'SOM',
    'South Sudan':'SSD','Sudan':'SDN','Tanzania':'TZA','Togo':'TGO','Tunisia':'TUN','Uganda':'UGA'
}
iso_to_name = {v: k for k, v in name_to_iso.items()}

# 2) Build a case/whitespace-normalized key to improve matches
def norm(s: pd.Series) -> pd.Series:
    return (s.astype(str)
              .str.strip()
              .str.replace(r'\s+', ' ', regex=True)
              .str.casefold())

# Normalized mapping dict (case-insensitive keys)
name_to_iso_norm = {k.casefold(): v for k, v in name_to_iso.items()}

# 3) Create a normalized key column for merged_long
merged_long["country_key"] = norm(merged_long["country"])

# 4) Fill ONLY missing ISO codes from country names
if "iso" not in merged_long.columns:
    merged_long["iso"] = merged_long["country_key"].map(name_to_iso_norm)
else:
    missing_mask = merged_long["iso"].isna() | (merged_long["iso"].astype(str).str.strip() == "")
    merged_long.loc[missing_mask, "iso"] = merged_long.loc[missing_mask, "country_key"].map(name_to_iso_norm)

# 5) Uppercase and clean ISO
merged_long["iso"] = merged_long["iso"].astype(str).str.upper().where(merged_long["iso"].notna(), other=pd.NA)

# 6) Standardize country names FROM iso where possible; otherwise keep original
merged_long["country"] = merged_long["iso"].map(iso_to_name).fillna(merged_long["country"].str.strip())

# 7) (Optional) sanity check: which still lack ISO?
unmapped = merged_long.loc[merged_long["iso"].isna(), ["country"]].drop_duplicates().sort_values("country")
print("Unmapped countries (no ISO after mapping):")
print(unmapped)

# 8) Clean up helper column
merged_long.drop(columns=["country_key"], inplace=True)


Unmapped countries (no ISO after mapping):
Empty DataFrame
Columns: [country]
Index: []


In [8]:
merged_long.tail(100)

,country,iso,year,variable,value,region,category
282913,Ethiopia,ETH,1998,VC.BTL.DETH,1040.0,EA,Security
282914,Ethiopia,ETH,1996,VC.BTL.DETH,52.0,EA,Security
282915,Ethiopia,ETH,1995,VC.BTL.DETH,29.0,EA,Security
282916,Ethiopia,ETH,1994,VC.BTL.DETH,75.0,EA,Security
282917,Ethiopia,ETH,1993,VC.BTL.DETH,36.0,EA,Security
...,...,...,...,...,...,...,...
283008,Uganda,UGA,1994,VC.BTL.DETH,149.0,EA,Security
283009,Uganda,UGA,1992,VC.BTL.DETH,72.0,EA,Security
283010,Uganda,UGA,1991,VC.BTL.DETH,857.0,EA,Security
283011,Uganda,UGA,1990,VC.BTL.DETH,626.0,EA,Security


## VARIABLE NAME MAPPING

In [9]:
import pandas as pd

# --- 1) Normalize each indicators object into {code: label} dicts ---
def to_code_label_map(obj):
    """
    Accepts either:
      - dict like {"NY.GDP.MKTP.CD": "GDP (current US$)", ...}
      - list/tuple like ["NY.GDP.MKTP.CD", ...]  -> maps to itself as label
    Returns a dict {code: label}.
    """
    if isinstance(obj, dict):
        return obj
    if isinstance(obj, (list, tuple, set)):
        return {code: str(code) for code in obj}
    raise TypeError("Indicators must be dict or list-like.")

econ_map = to_code_label_map(economic_indicators)
demo_map = to_code_label_map(demographic_indicators)
pol_map  = to_code_label_map(political_indicators)
sec_map  = to_code_label_map(security_indicators)

# --- 2) Merge all into a single code→label dictionary (later items overwrite earlier on conflicts) ---
code_to_label = {}
code_to_label.update(econ_map)
code_to_label.update(demo_map)
code_to_label.update(pol_map)
code_to_label.update(sec_map)

# --- 3) Add the plain-language label next to the code ---
# If a code is missing in the map, keep the code itself as the label (no NaNs).
merged_long["variable_label"] = merged_long["variable"].map(code_to_label).fillna(merged_long["variable"])

# (Optional) place the label column right after 'variable' for readability
cols = list(merged_long.columns)
if "variable" in cols and "variable_label" in cols:
    cols.remove("variable_label")
    cols.insert(cols.index("variable") + 1, "variable_label")
    merged_long = merged_long[cols]

# --- 4) Quick check of any codes that weren’t labeled (so you can extend your indicator dicts) ---
unmapped_codes = sorted(set(merged_long["variable"].unique()) - set(code_to_label.keys()))
print("Unmapped variable codes:", unmapped_codes[:50], f"... total {len(unmapped_codes)}")


Unmapped variable codes: [] ... total 0


In [10]:
merged_long.head(100)

,country,iso,year,variable,variable_label,value,region,category
0,Algeria,DZA,2024,NY.GDP.MKTP.CD,GDP (current US$),2.636198e+11,NA,Economic
1,Algeria,DZA,2023,NY.GDP.MKTP.CD,GDP (current US$),2.476262e+11,NA,Economic
2,Algeria,DZA,2022,NY.GDP.MKTP.CD,GDP (current US$),2.256385e+11,NA,Economic
3,Algeria,DZA,2021,NY.GDP.MKTP.CD,GDP (current US$),1.862312e+11,NA,Economic
4,Algeria,DZA,2020,NY.GDP.MKTP.CD,GDP (current US$),1.648734e+11,NA,Economic
...,...,...,...,...,...,...,...,...
95,"Egypt, Arab Rep.",EGY,1994,NY.GDP.MKTP.CD,GDP (current US$),5.189798e+10,NA,Economic
96,"Egypt, Arab Rep.",EGY,1993,NY.GDP.MKTP.CD,GDP (current US$),4.657863e+10,NA,Economic
97,"Egypt, Arab Rep.",EGY,1992,NY.GDP.MKTP.CD,GDP (current US$),4.185599e+10,NA,Economic
98,"Egypt, Arab Rep.",EGY,1991,NY.GDP.MKTP.CD,GDP (current US$),3.738784e+10,NA,Economic


## Storing as CSV

In [11]:
import os

# Define the output directory
output_dir = "/home/jean-axel/labserver/frontend/assets/exports/csv"

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the output file path
output_file = os.path.join(output_dir, "merged_long.csv")

# Save the dataframe to CSV
merged_long.to_csv(output_file, index=False)

print(f"✅ DataFrame saved to: {output_file}")


✅ DataFrame saved to: /home/jean-axel/labserver/frontend/assets/exports/csv/merged_long.csv


## Connecting to the Superset Database

### Library Imports

In [ ]:
pip install sqlalchemy psycopg2 pandas
from sqlalchemy import create_engine
import pandas as pd

### Operational Functions

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

def connect_to_superset_db(user, password, host, port, dbname):
    """
    Establish connection to the Superset (PostgreSQL) database.
    """
    try:
        engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")
        return engine
    except Exception as e:
        raise ConnectionError(f"Failed to connect: {e}")

def upsert_table(df, table_name, key_column, engine):
    """
    Insert or update a table based on a key column.

    If the table exists, update records based on key_column.
    If it doesn't exist, create and insert the table.

    Parameters:
    - df: DataFrame to insert
    - table_name: Target table name
    - key_column: Unique identifier column for merging
    - engine: SQLAlchemy engine
    """
    from sqlalchemy import text

    # Check if table exists
    with engine.connect() as conn:
        result = conn.execute(text(
            "SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = :table)"
        ), {"table": table_name})
        table_exists = result.scalar()

    if not table_exists:
        print(f"Table '{table_name}' does not exist. Creating new table...")
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    else:
        print(f"Table '{table_name}' exists. Updating rows based on '{key_column}'...")
        existing_df = pd.read_sql_table(table_name, con=engine)

        # Merge logic: prioritize new data
        updated_df = pd.concat([existing_df, df]).drop_duplicates(subset=[key_column], keep='last')

        # Replace table with updated content
        updated_df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print(f"Table '{table_name}' successfully updated.")



## Inserting in database

In [66]:
import pandas as pd
from urllib.parse import quote_plus
from sqlalchemy import create_engine, MetaData, Table, Column, Text, String, Integer, Float
from sqlalchemy import text

# --------- CONNECT ----------
def connect_to_superset_db(user: str, password: str, host: str, port: str, dbname: str):
    pw = quote_plus(password.strip())  # URL-encode specials like '#'
    url = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{dbname}?sslmode=require"
    return create_engine(url, pool_pre_ping=True, pool_size=5, max_overflow=5)

# --------- PREP DF (no matching; just clean types) ----------
def prep_df_for_append(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d.columns = [c.strip().lower() for c in d.columns]

    # Ensure expected columns exist, fill missing with NA
    must_have = ["country", "iso", "year", "variable", "variable_label", "value", "region", "category"]
    for c in must_have:
        if c not in d.columns:
            d[c] = pd.NA

    # Type coercions
    d["iso"] = d["iso"].astype(str).str.upper().str.strip()
    d["country"] = d["country"].astype(str).str.strip()
    d["variable"] = d["variable"].astype(str).str.strip()
    d["variable_label"] = d["variable_label"].astype(str)
    d["region"] = d["region"].astype(str)
    d["category"] = d["category"].astype(str)

    d["year"] = pd.to_numeric(d["year"], errors="coerce").astype("Int64")
    d["value"] = pd.to_numeric(d["value"], errors="coerce")

    # Keep only the columns we’re going to store, in order
    cols = ["country", "iso", "year", "variable", "variable_label", "value", "region", "category"]
    return d[cols]

# --------- ENSURE TABLE (no unique constraints, iso indexed) ----------
def ensure_append_table(engine, table_name: str = "wbdata"):
    metadata = MetaData(schema="public")
    table = Table(
        table_name, metadata,
        # Surrogate PK so appends never collide:
        Column("id", Integer, primary_key=True, autoincrement=True),
        Column("country", Text),
        Column("iso", String(3), nullable=False),
        Column("year", Integer),
        Column("variable", String(64)),
        Column("variable_label", Text),
        Column("value", Float),
        Column("region", String(16)),
        Column("category", String(32)),
        schema="public"
    )
    metadata.create_all(engine)  # creates if missing

    with engine.begin() as conn:
        # Drop any old unique constraints that could block appends
        conn.execute(text(f"""
        DO $$
        DECLARE r record;
        BEGIN
          FOR r IN SELECT conname
                   FROM pg_constraint
                   WHERE conrelid = 'public.{table_name}'::regclass
                     AND contype IN ('u','p')
          LOOP
            EXECUTE format('ALTER TABLE public.{table_name} DROP CONSTRAINT %I', r.conname);
          END LOOP;
        END$$;"""))

        # Make iso the "key column" via an index (fast lookup, not unique)
        conn.execute(text(f"CREATE INDEX IF NOT EXISTS {table_name}_iso_idx ON public.{table_name}(iso);"))

    return table

# --------- APPEND (no matching) ----------
def append_all(df: pd.DataFrame, engine, table_name: str = "wbdata", chunksize: int = 5000):
    # Count before
    with engine.begin() as conn:
        before = conn.execute(text(f"SELECT COUNT(*) FROM public.{table_name}")).scalar()

    # Append
    df.to_sql(
        table_name,
        engine,
        schema="public",
        if_exists="append",
        index=False,
        method="multi",
        chunksize=chunksize
    )

    # Count after
    with engine.begin() as conn:
        after = conn.execute(text(f"SELECT COUNT(*) FROM public.{table_name}")).scalar()
    print(f"✅ Appended {after - before} rows (total now {after}).")



In [68]:
# ================== RUN ==================
data = merged_long  # your assembled DataFrame
df = prep_df_for_append(pd.DataFrame(data))

engine = connect_to_superset_db(
    user="postgres.ogfoykakuramtzwiueix",
    password="Wk5lsx9WVAhJHWIo",  # make sure this is the actual DB password
    host="aws-0-eu-north-1.pooler.supabase.com",
    port="5432",
    dbname="postgres"
)

ensure_append_table(engine, table_name="wbdata")
append_all(df, engine, table_name="wbdata")


✅ Appended 281605 rows (total now 281605).


# Create Local Database

In [28]:
!pip install geopandas sqlalchemy psycopg2-binary shapely

import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine, text
import os
from pathlib import Path
from IPython.display import display

Defaulting to user installation because normal site-packages is not writeable


## Functions

In [31]:
# Required imports
import os
from pathlib import Path
import geopandas as gpd
from sqlalchemy import create_engine, text

# -------------------------
# 1. Create PostgreSQL connection string
# -------------------------
def create_postgres_connection_string(user, password, host, port, dbname):
    """
    Create PostgreSQL connection string
    """
    from urllib.parse import quote_plus
    password_encoded = quote_plus(password)
    return f"postgresql+psycopg2://{user}:{password_encoded}@{host}:{port}/{dbname}"

# -------------------------
# 2. Convert Shapefile (DBF + SHP) to PostgreSQL
# -------------------------
def dbf_to_postgres_with_geometry(dbf_file_path, db_connection_string, 
                                  table_name="africa_boundaries", schema="public"):
    """
    Convert DBF file (with SHP geometry) to PostgreSQL
    """
    try:
        dbf_path = Path(dbf_file_path)
        base_path = dbf_path.parent / dbf_path.stem
        shp_file_path = f"{base_path}.shp"

        print(f"📁 DBF file: {dbf_file_path}")
        print(f"🔍 Looking for SHP file: {shp_file_path}")

        # Ensure shapefile exists
        if not os.path.exists(shp_file_path):
            print("❌ SHP file not found! Falling back to DBF-only.")
            return False

        # Load shapefile
        print("📖 Reading shapefile...")
        gdf = gpd.read_file(shp_file_path)

        print(f"📊 Features: {len(gdf)}")
        print(f"📍 CRS: {gdf.crs}")
        display(gdf.head(3))

        # Connect to PostgreSQL
        engine = create_engine(db_connection_string)
        with engine.connect() as conn:
            db_version = conn.execute(text("SELECT version()")).scalar()
            print(f"✅ Connected to PostgreSQL: {db_version}")

        # Export to PostgreSQL with geometry
        gdf.to_postgis(
            name=table_name,
            con=engine,
            schema=schema,
            if_exists='replace',  # Change to 'append' if needed
            index=False,
            chunksize=1000
        )

        print(f"✅ Exported {len(gdf)} rows to {schema}.{table_name}")

        # Verification
        with engine.connect() as conn:
            row_count = conn.execute(text(f"SELECT COUNT(*) FROM {schema}.{table_name}")).scalar()
            print(f"🔍 Verification: {row_count} rows in DB")

        return True

    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback; traceback.print_exc()
        return False

# -------------------------
# 3. Fallback: DBF only (no geometry)
# -------------------------
def dbf_only_to_postgres(dbf_file_path, db_connection_string, 
                         table_name="africa_data", schema="public"):
    """
    Convert DBF file (no geometry) to PostgreSQL
    """
    try:
        print("📖 Reading DBF only...")
        gdf = gpd.read_file(dbf_file_path)
        print(f"📊 Records: {len(gdf)}")
        display(gdf.head(3))

        engine = create_engine(db_connection_string)
        gdf.to_sql(
            name=table_name,
            con=engine,
            schema=schema,
            if_exists='replace',
            index=False,
            chunksize=1000
        )

        print(f"✅ Exported {len(gdf)} rows to {schema}.{table_name}")
        return True

    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback; traceback.print_exc()
        return False



## CONNECTING LOCALLY

### Install Libraries

In [34]:
import pandas as pd
from sqlalchemy import create_engine


### Connection instance

In [36]:

# Database config
DB_CONFIG = {
    "user": "postgres",  # or jean_axel if you prefer
    "password": "axeldevlab1",  # your actual postgres password
    "host": "localhost",
    "port": "5432",
    "dbname": "axeldevlabmain"
}

# Connection string
engine = create_engine(
    f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
)




### Add data to the database

In [37]:
# Path to your CSV
csv_file = "/home/jean-axel/labserver/frontend/assets/exports/csv/merged_long.csv"

# Load CSV into pandas
df = pd.read_csv(csv_file)

# Ensure 'iso' column is string (to match ISO in shapefile table)
df['iso'] = df['iso'].astype(str)

# Push to PostgreSQL
df.to_sql("merged_long", engine, if_exists="replace", index=False)

print("✅ CSV imported as table 'merged_long'")

✅ CSV imported as table 'merged_long'
